# Contrôle des joints du bras Kinova (Kinova arm control)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/AIResearchLab/foundations-of-robotics-labs/blob/master/3-kinematics/03-3dof-joint-control.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/AIResearchLab/foundations-of-robotics-labs/master?filepath=3-kinematics/03-3dof-joint-control.ipynb)

Ce script Python permet de contrôler les trois premiers degrés de liberté du bras robotique Gen3 Lite en envoyant des commandes conjointes. Il fonctionne autant avec le bras simulé que le bras réel.

(en) This Python script allows to control the first three degrees of freedom of the Kinova Gen3 Lite robotic arm by sending joint commands. It works with both the simulated and the real arm.

> The purpose of this notebook is to learn how to control the Kinova Gen3 Lite robotic arm using the first three joints.

## Run the Gen3 Lite Control Application

> #### **Make sure to run the Gen3 Lite Control Application before running this notebook.**

Start with the simulation environment, and then move to the real robot. In a terminal, run the following command:

```bash
# in a terminal on the PC
roslaunch kortex_gazebo spawn_kortex_robot.launch arm:=gen3_lite

############################################################
# # !!!! DO NOT RUN THE SIM AND REAL AT THE SAME TIME !!!! #
############################################################

# in a terminal on the PC
# make sure the robot is connected to the PC via USB
roslaunch kortex_driver kortex_driver.launch arm:=gen3_lite
```

### Import des paquets Python (Import Python ROS packages)

Just in case, let's start by importing the necessary Python packages for the ROS environment.

In [ ]:
"""setting up ros enrionment variables"""

# this set of lines is to make sure that the python scripts can find the ROS libraries
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages')
sys.path.append('/usr/lib/python3/dist-packages')

import os

#####################################################################
# # !!!! DON'T FORGET TO CHANGE THIS TO THE ROBOT'S IP ADDRESS !!!! #
#####################################################################

# this should be the localhost address as below
ROBOT_HOSTNAME = '127.0.0.1' # localhost
CONSOLE_HOSTNAME = '127.0.0.1' # localhost

# the following lines set up the networking variables for the scripts to run properly
os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROBOT_HOSTNAME)
os.environ['ROS_HOSTNAME'] = CONSOLE_HOSTNAME

In [ ]:
from os import environ
import numpy as np
import rospy
import time
from time import sleep
import math
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
from mobile_manip.srv import ReachValues, GetValues, ReachName

###  Fonction pour envoyer des valeurs au robot simulé dans Gazebo

### (en) Function to send values to the simulated robot in Gazebo

In [ ]:
def send_joint_angles(ang):
    # Create the list of angles
    #rospy.wait_for_service('/mobile_manip/reach_joints', timeout=20)
    reach_joints_angles = rospy.ServiceProxy('/mobile_manip/reach_joints', ReachValues)
    return reach_joints_angles(ang)

### Fonction de simplification pour le projet 2 (Simplification function for project 2)

In [ ]:
# Définissez les valeurs des 3 joints a controler en degrés
# (en) Define the values of the 3 joints to control in degrees
def dof3control(joint1, joint2, joint3):
    cmd = np.array([joint1, joint2, joint3, 0, 0, 0])
    send_joint_angles(cmd)

### Fonction pour la cinématique directe (Function for direct kinematics)

In [ ]:
# Accepte les valeurs theta, d, a, alpha. Les angles doivent être en radians.
# (en) Accepts theta, d, a, alpha values. Angles must be in radians.
def dh(theta,d,a,alpha):
    return np.array([
        [np.cos(theta), -1*np.sin(theta)*np.cos(alpha),    np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta),    np.cos(theta)*np.cos(alpha), -1*np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [            0,                  np.sin(alpha),                  np.cos(alpha),               d],
        [            0,                              0,                              0,               1]
    ])

### Calculs de cinematique inverse (Inverse kinematics calculations)

In [ ]:
def ik(x,y,z):

    # Calcul des angles des joints
    # (en) Calculation of joint angles
    px = x
    py = y
    pz = z

    joint1 = 0
    joint2 = 0
    joint3 = 0

    return np.array([joint1,joint2,joint3])

In [ ]:
# Mouvement vers la position de départ.
# (en) Movement to the starting position.
dof3control(0,0,0)

In [ ]:
# Execution des angles pour le point X de la trajectoire.
# (en) Execution of angles for point X of the trajectory.

P1 = ik(0,0,0)/np.pi*180
print(P1)
dof3control(P1[0],P1[1],P1[2])

In [ ]:
# Vérification de la pose finale
# (en) Verification of the final pose
offsets_cart = [0.0, 0.0, 0.0]
rospy.wait_for_service('/mobile_manip/get_cartesian')
get_pose = rospy.ServiceProxy('/mobile_manip/get_cartesian', GetValues)
res = get_pose()
res_arr = np.array(res.val)
res_arr[:3] = (res_arr[:3]-offsets_cart)*1000.0
print(res_arr)